In [1]:
import csv
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import graph_tool as gt
import graph_tool.util as gu
import os
import pickle

from dash import Dash, dcc, html, Input, Output, State, ctx
from graph_tool import GraphView
from jupyter_dash import JupyterDash

In [2]:
def get_filtered_network(node, sample, weight_filter_up, weight_filter_low, checklist_show):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    g = gr
    v_list = gu.find_vertex(g, g.vp.name, node)
    if len(v_list) != 0:
        v = v_list[0]
    else:
        ed = gu.find_edge(g, g.ep.rem, node)[0]
        v = ed.source()

    nodes = []
    edges= []
    if g.vp.name[v].startswith("E") and sample != "score" and sample != "p_value":
        nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': g.vp.start[v], 'end': g.vp.end[v], 'expr': g.vp[sample][v], 'chr': g.vp.chr[v]}, 'classes':'black'})
    else:
        nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': g.vp.start[v], 'end': g.vp.end[v], 'chr': g.vp.chr[v]}, 'classes':'black'})
    for e, w in zip(v.all_edges(), v.all_neighbors()):
            if (weight_filter_up != None and g.edge_properties[sample][e] > weight_filter_up) or (weight_filter_low != None and g.edge_properties[sample][e] < weight_filter_low):
                continue
            else:
                if g.edge_properties[sample][e] >= 0: color = "green"
                else: color = "red"
                edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.edge_properties[sample][e], 'rem': g.ep.rem[e]}, 'style':{'width':abs(g.edge_properties[sample][e])*5, 'line-color': color}})
            if g.vp.name[w].startswith("E") and sample != "score" and sample != "p_value":
                nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w], 'start': g.vp.start[w], 'end': g.vp.end[w], 'expr': g.vp[sample][w], 'chr': g.vp.chr[v]}, 'classes':'DodgerBlue'})
            else:
                nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w], 'start': g.vp.start[w], 'end': g.vp.end[w], 'chr': g.vp.chr[v]}, 'classes': 'DodgerBlue'})
    if 'Show weights' not in checklist_show:
        for e in edges: e.get('data').pop('label')
    cyto_list = nodes+edges
    return cyto_list

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Div([
        html.H1("REMnet", style={"textAlign":"center", "text-decoration":"underline", "fontSize":40, 'font-family':'Consolas'})
    ]),
    # html.Div([
    #     html.H3("General Analysis", style={"text-decoration":"underline", "fontSize":20, 'font-family':'Consolas'}),
    #     "Choose centrality algorithm",
    #     dcc.Dropdown(
    #         id='dropdown-cen-algo',
    #         value=[],
    #         clearable=False,
    #         options=[
    #             {'label': name, 'value': name}
    #             for name in ["betweenness", "closeness"]
    #         ]
    #     ),
    #     'Choose clustering algorithm',
    #     dcc.Dropdown(
    #         id='dropdown-clu-algo',
    #         value=[],
    #         clearable=False,
    #         options=[
    #             {'label': name, 'value': name}
    #             for name in ["local", "global", "extended"]
    #         ]
    #     ),
    #     html.Div(id='out-text'),
    #     dbc.Button('Run', id= 'run-algo', n_clicks=0, outline=True, color="primary", className="me-1")
    # ], style={'height': '230px'}
    # ),



    dcc.Loading(
        children=[
            html.Div([
                html.Div([
                    html.Div([
                        html.Div([
                            html.Div([
                                # html.H3("Detailed Analysis", style={"text-decoration":"underline", "fontSize":20, 'font-family':'Consolas'}),
                                "Select celltype",
                                dcc.Dropdown(
                                    id='dropdown-graph',
                                    value=os.listdir('../data/graph-tool/')[0][:-3],
                                    clearable=False,
                                    options=[
                                        {'label': name[:-3], 'value': name[:-3]}
                                        for name in os.listdir('../data/graph-tool/')
                                    ]
                                )
                            ], style={'display': 'block', 'height': '80px'}
                            ),
                            html.Div([
                                "Select sample",
                                dcc.Dropdown(
                                    id= 'dropdown-sample',
                                    value= '',
                                    clearable=False,
                                    options=[]
                                )
                            ], style={'display': 'block', 'height': '80px'}
                            ),
                            html.Div([
                                "Select gene",
                                dcc.Dropdown(
                                id='dropdown-gen',
                                clearable=False,
                                options=[]
                                )
                            ],style={'display': 'block', 'height': '80px'}
                            ),
                            html.Div([
                                html.P('Enter a CREM ID from 1 to 365.286: '),
                                dcc.Input(id='input-crem', type='number', value = 1, step=1, min=1, max=365286, debounce=True),
                            ], style={'height': '80px'}
                            ),
                            html.Div([
                                html.P('Enter a REM ID from 1 to 2.404.862: '),
                                dcc.Input(id='input-rem', type='number', value = 1, step=1, min=1, max=2404862, debounce=True),
                            ], style={'height': '80px'}
                            ),
                        ], style={'display': 'inline-block', 'padding-left' : '10px', 'padding-right' : '10px', 'padding-top' : '10px', 'padding-bottom' : '10px'}),
                    ], style={"border":"1px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),

                    html.Div([
                        html.Div([
                            html.Div([
                                html.P("Enter gene degree filter: "),
                                dcc.Slider(id='filter-gen', min=0, max=108, step=1, value=0, marks=None, tooltip={"placement": "bottom", "always_visible": True}),
                            ], style={'height': '80px'}
                            ),

                            html.Div([
                                "Set an upper bond weight filter:  ",
                                dcc.Input(id='filter-weight-up', type='number', debounce=True),
                                html.Div(id= 'filter-text-up')
                            ], style={'height': '80px'}),
                            html.Div([
                                "Set a lower bond weight filter:  ",
                                dcc.Input(id='filter-weight-low', type='number', debounce=True),
                                html.Div(id= 'filter-text-low')
                            ], style={'height': '80px'}),
                        ], style={'display': 'inline-block', 'padding-left' : '10px', 'padding-right' : '10px', 'padding-top' : '10px', 'padding-bottom' : '10px'}),
                    ], style={"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),

                    dbc.Button('Reset filter', id= 'reset-filter', n_clicks=0, outline=True, color="primary", className="me-1")

                ], style={'display': 'inline-block', 'vertical-align': 'top', 'width': '14%'}
                ),
                html.Div([
                    html.Div([
                        cyto.Cytoscape(
                            id='network-graph',
                            style={'width': '100%', 'height': '700px'},
                            layout={'name': 'circle', 'avoidOverlap': 'true', 'nodeDimensionsIncludeLabels': 'true'},
                            elements=[],
                            stylesheet=[
                    #             # Group selectors
                                {
                                    'selector': 'node',
                                    'style': {
                                        'content': 'data(label)'
                                    }
                                },
                                {
                                    'selector': 'edge',
                                    'style': {
                                        'curve-style': 'bezier',
                                        'content': 'data(label)'
                                    }
                                },
                                {
                                    'selector': '.DodgerBlue',
                                    'style': {
                                        'background-color': 'DodgerBlue',
                                        'line-color': 'DodgerBlue'
                                    }
                                },
                                {
                                    'selector': '.black',
                                    'style': {
                                        'background-color': 'black',
                                        'line-color': 'black'
                                    }
                                },
                                {
                                    'selector': '.triangle',
                                    'style': {
                                        'shape': 'triangle'
                                    }
                                }
                            ]
                        ),
                    ]),
                    html.Div([
                        html.P(id= 'mouse-hover', style={'padding-left' : '10px', 'padding-right' : '10px', 'padding-top' : '10px', 'padding-bottom' : '7px'}),
                    ], style={"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'float':'left','display': 'inline-block'}),
                    dcc.Checklist(options=['Show weights'], id='checklist-show', value=['Show weights'])
                ], style={ 'vertical-align': 'top', 'display': 'inline-block', 'width': '85%', 'height': '800px', 'float': 'right'} #'border':'2px black solid', 'borderRadius':'20px',
                ),
                ]),
            ],type='circle',
        ),
    dcc.Store(id='gen-or-rem'),
    dcc.Store(id='gen-degrees'),

], style={'margin-left' : '10px', 'margin-right' : '10px', 'margin-top' : '10px', 'margin-bottom' : '10px'})

# @app.callback(Output('out-text', 'children'),
#               Input('run-algo', 'n_clicks'),
#               State('dropdown-cen-algo', 'value'))
# def run_cen_algo(n_clicks, value):
#     return 'The input value was "{}" and the button has been clicked {} times'.format(value, n_clicks)

@app.callback(Output('dropdown-sample', 'options'),                     # disable sample dropdown if allCelltypeGraph or only one sample in celltype
              Output('dropdown-sample' ,'value'),
              Output('gen-degrees', 'data'),
              Input('dropdown-graph', 'value'))
def update_file(celltype):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    gr = gt.load_graph(f"../data/graph-tool/{celltype}.gt")
    gen_degrees = {gr.vp.name[v] : len(gr.get_in_edges(v)) for v in gr.vertices() if gr.vp.name[v].startswith("E")}
    sample_list = [s for s in gr.edge_properties if s not in ("celltype", "celltypeID", "rem")]
    val_sample = sample_list[0]
    return sample_list, val_sample, gen_degrees

@app.callback(Output('dropdown-gen', 'options'),            # dauert zu lange, vllt sideeffect, aber eig überprüft dass kein sideeffect
              Output('dropdown-gen', 'value'),
              Input('gen-degrees', 'data'),
              Input('dropdown-graph', 'value'),
              Input('filter-gen', 'value'))
def update_gen_dropdown(gen_degrees, celltype, gen_filter):
    if ctx.triggered_id == 'dropdown-graph':
        gen_list = list(gen_degrees.keys())
        val_gen = gen_list[0]
        return gen_list, val_gen
    elif ctx.triggered_id == 'filter-gen' and gen_filter is not None:
        gen_list = [k for k, v in gen_degrees.items() if v >= gen_filter]
        return gen_list, gen_list[0]
    else:
        gen_list = list(gen_degrees.keys())
        val_gen = gen_list[0]
        return gen_list, val_gen


@app.callback(Output('network-graph', 'elements'),                      # start value for rem or catch Nonetype error
              Output('gen-or-rem', 'data'),
              Input('dropdown-gen', 'value'),
              Input('input-rem', 'value'),
              Input('input-crem', 'value'),
              Input('dropdown-sample', 'value'),
              Input('filter-weight-up', 'value'),
              Input('filter-weight-low', 'value'),
              Input('checklist-show', 'value'),
              Input('gen-or-rem', 'data'),
              State('network-graph', 'elements'))
def update_elements(gen, rem_int, crem_int, sample, filter_up, filter_low, checklist_show, gen_or_rem, elements):
    if ctx.triggered_id == 'dropdown-gen':
        return get_filtered_network(gen, sample, filter_up, filter_low, checklist_show), gen
    elif ctx.triggered_id == 'input-rem':
        rem = f'REM{rem_int:>07}'
        return get_filtered_network(rem, sample, filter_up, filter_low, checklist_show), rem
    elif ctx.triggered_id == 'input-crem':
        crem = f'CREM{crem_int:>07}'
        return get_filtered_network(crem, sample, filter_up, filter_low, checklist_show), crem
    elif gen_or_rem is not None:
        return get_filtered_network(gen_or_rem, sample, filter_up, filter_low, checklist_show), gen_or_rem
    else:
        return get_filtered_network(gen, sample, filter_up, filter_low, checklist_show), gen

@app.callback(Output('filter-weight-up', 'value'),
              Output('filter-weight-low', 'value'),
              Output('filter-gen', 'value'),
              Input('reset-filter', 'n_clicks'))
def reset_filter(n_clicks):
    return None, None, None


@app.callback(Output('mouse-hover', 'children'),
              Input('network-graph', 'tapNodeData'))
def node_hover(data):
    if data:
        if len(data) == 6:
            return html.Strong('Node info'), html.Br(), f'Name: {data["label"]}', html.Br(), f'Expression: {data["expr"]}', html.Br(), f'Chromosome: {data["chr"]}', html.Br(),f'Start: {data["start"]} ', html.Br(),f'End: {data["end"]}'
        else:
            return html.Strong('Node info'), html.Br(), f'Name: {data["label"]} ', html.Br(), f'Chromosome: {data["chr"]}', html.Br(),f' Start: {data["start"]} ', html.Br(),f' End: {data["end"]}'
    else:
        return html.Strong('Node info'), html.Br()

# app.run_server(host='0.0.0.0', port=8050, debug=False)
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [4]:
# import matplotlib.pyplot as plt
#
# weights = [g.ep.standDnase1Log2[e] for e in g.edges()]
# plt.hist(weights)
# plt.xlabel('weight')
# plt.show()

In [5]:
# graph_pickle_safe = open("graph_pickle", "wb")
# gen_list = [{'label': name, 'value': name} for name in [a for a in list(gr.vp.name) if a.startswith("E")]]
# pickle.dump(gr, graph_pickle_safe)
# graph_pickle_safe.close()

In [6]:
# g = gt.load_graph('../data/graph-tool/allCelltypeGraph.gt')
# v = gu.find_vertex(g, g.vp.name, 'ENSG00000278267')[0]
# cyto_list = []
# graph_type = 'allCelltypeGraph.gt'
# nodes = []
# edges= []
# # vfilt = g.new_vertex_property('bool')
# # vfilt[v] = True
# nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v]}, 'classes':'green'})
# for e, w in zip(v.in_edges(), v.in_neighbors()):
#     # vfilt[w] = True
#     nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w]}, 'classes': 'red triangle'})
#     if graph_type != "allCelltypeGraph.gt":
#         edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.standDnase1Log2[e]}})
#     else:
#         if g.ep.score[e] >= 0: color = "green"
#         else: color = "red"
#         edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.score[e]}, 'style':{'width':g.ep.score[e]*5, 'color': color}})
# # sub = GraphView(g, vfilt)
# cyto_list = nodes+edges
# cyto_list